In [10]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00


In [28]:
# === Fast Inference API (Optimized):
#     YOLO → (조건부 전체이미지 / 소프트 크롭)
#     → ResNet(1324) 최종 Top-5 (dl_idx 계산식 기반) ===
# -*- coding: utf-8 -*-
import os
import json
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms

from ultralytics import YOLO
from PIL import Image

# -------------------------------------------------
# 0) 경로 / 기본 설정
# -------------------------------------------------
DRIVE = "/content/drive/MyDrive"

# 테스트용 기본 이미지 경로 (필요하면 이 값만 바꿔서 사용)
IMG_PATH = "/content/drive/MyDrive/캡스톤_원천_데이터/TS_34_단일.zip/K-009272/K-009272_0_1_0_0_70_000_200.png"

BEST_YOLO      = os.path.join(DRIVE, "best.pt")
RESNET_1324_PT = os.path.join(DRIVE, "best_model_generalized.pth")
CLASS_JSON_1K  = os.path.join(DRIVE, "pill_label_path_sharp_score.json")
CLASS_JSON_324 = os.path.join(DRIVE, "class_mapping_from_cache_1324.json")

for p in [BEST_YOLO, RESNET_1324_PT]:
    assert os.path.exists(p), f"가중치 파일 없음: {p}"

YOLO_CONF  = 0.25
YOLO_IOU   = 0.45
YOLO_IMGSZ = 640

CROP_SIZE        = 224
MIN_BOX_SIDE_PX  = 40
FULL_IMAGE_AREA_RATIO_THRESHOLD = 0.65
SQUARE_SCALE     = 1.3

NUM_CLASSES  = 1324
LABEL_OFFSET = 1000

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# -------------------------------------------------
# 1) 라벨 맵 로드 (class_idx → K-코드)
#    그리고 최종 dl_idx는 "K-xxxxxx"의 뒤 6자리 정수 - 1 로 계산
#    ⚠️ 이 부분도 한 번만 로드하도록 전역에서 실행
# -------------------------------------------------
def load_label_map_generic(json_path):
    """
    pill_label_path_sharp_score.json / class_mapping_from_cache_1324.json 을 읽어서
    class_idx → K코드(K-xxxxxx) 매핑을 만든다.
    값이 경로/파일명일 경우 파일명에서 K코드를 파싱해서 사용.
    """
    if not json_path or not os.path.exists(json_path):
        return {}
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    if isinstance(data, dict) and "label_to_kcode" in data:
        data = data["label_to_kcode"]

    out = {}
    if isinstance(data, dict):
        for k, v in data.items():
            try:
                key_int = int(k)
            except:
                continue

            val_str = str(v)
            base = os.path.basename(val_str)   # 예: "K013101_0_2_0_1_90_020_200.png"
            first = base.split("_")[0]         # 예: "K013101" 또는 "K-009272"

            if first.startswith("K-") and len(first) == 8 and first[2:].isdigit():
                # 이미 "K-009272" 같은 형식
                kcode = first
            elif first.startswith("K") and len(first) == 7 and first[1:].isdigit():
                # "K013101" → "K-013101"
                kcode = "K-" + first[1:]
            else:
                # 그 외에는 그냥 값 그대로 사용 (fallback)
                kcode = first

            out[key_int] = kcode

    return out

LABEL_MAP_1K  = load_label_map_generic(CLASS_JSON_1K)   # 0..999 → K코드
LABEL_MAP_324 = load_label_map_generic(CLASS_JSON_324)  # 0..323 → K코드 (1000~1323용)

def class_idx_to_kcode(global_idx: int) -> str:
    """
    모델의 global class index → K코드(K-xxxxxx)로 변환
    """
    if global_idx < LABEL_OFFSET:
        return LABEL_MAP_1K.get(global_idx, f"imagenet_{global_idx}")
    local = global_idx - LABEL_OFFSET
    return LABEL_MAP_324.get(local, f"unknown_{local}")

def kcode_to_dl_idx(kcode: str) -> str:
    """
    K코드에서 dl_idx 계산:
      예) "K-009272" → 뒤 6자리 009272 → 9272 - 1 = 9271
    매핑 파일 없이 규칙으로만 계산.
    """
    if len(kcode) >= 7:
        tail = kcode[-6:]  # 끝 6자리
        if tail.isdigit():
            val = int(tail)
            dl_val = val - 1
            if dl_val >= 0:
                return str(dl_val)
    # 실패 시 그냥 원래 문자열 반환 (디버그용)
    return kcode

def idx_to_dl_idx(global_idx: int) -> str:
    """
    class index → K코드 → dl_idx 로 변환
    """
    kcode = class_idx_to_kcode(global_idx)
    return kcode_to_dl_idx(kcode)

# -------------------------------------------------
# 2) ResNet 1324 모델 + 전처리 (전역 1회 로드)
# -------------------------------------------------
def build_resnet_1324(num_classes=NUM_CLASSES, model_path=RESNET_1324_PT):
    model = models.resnet152(weights=None)
    in_f = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(in_f, num_classes)
    )

    state = torch.load(model_path, map_location="cpu")
    if isinstance(state, dict):
        if "model_state_dict" in state:
            state = state["model_state_dict"]
        elif "model" in state:
            state = state["model"]
    missing, unexpected = model.load_state_dict(state, strict=False)
    if missing or unexpected:
        print(f"ℹ️ state_dict load: missing={len(missing)}, unexpected={len(unexpected)}")

    model.to(DEVICE)
    model.eval()

    # GPU면 half precision으로 살짝 더 빠르게
    if DEVICE.type == "cuda":
        model.half()

    return model

# 🔥 전역에서 한 번만 생성해서 재사용
RESNET_MODEL = build_resnet_1324()

# 전역 transform (이미지 → 텐서)
base_transform = transforms.Compose([
    transforms.Resize((CROP_SIZE, CROP_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

def preprocess_pil(pil_img):
    t = base_transform(pil_img)
    # 모델이 half면 입력도 half로 맞추기
    if DEVICE.type == "cuda":
        return t.half()
    return t

@torch.no_grad()
def predict_resnet_batch(pil_imgs, topk=5):
    """
    전역 RESNET_MODEL 사용.
    pil_imgs: [PIL.Image, ...]
    """
    if not pil_imgs:
        return []
    xs = [preprocess_pil(im) for im in pil_imgs]
    x = torch.stack(xs).to(DEVICE)

    # 이미 half로 올려놨으면 autocast 필요 없음 → 오버헤드 줄이기
    if DEVICE.type == 'cuda':
        logits = RESNET_MODEL(x)
    else:
        with torch.amp.autocast(device_type='cpu'):
            logits = RESNET_MODEL(x)

    probs  = F.softmax(logits, dim=1)
    topk_prob, topk_idx = torch.topk(probs, topk, dim=1)

    all_results = []
    for i in range(probs.shape[0]):
        res_i = []
        for p, idx in zip(topk_prob[i].tolist(), topk_idx[i].tolist()):
            res_i.append({
                "idx": int(idx),
                "prob": float(p),
            })
        all_results.append(res_i)
    return all_results

# -------------------------------------------------
# 3) YOLO 감지 + 크롭 생성 (전역 YOLO 1회 로드, 메모리 상 처리)
# -------------------------------------------------
# YOLO도 전역에서 한 번만 로드
YOLO_DEVICE = 0 if DEVICE.type == "cuda" else "cpu"
YOLO_MODEL = YOLO(BEST_YOLO)  # ultralytics가 내부에서 device 인자에 따라 처리

def square_crop_from_bbox(pil_img, xyxy, scale=1.3):
    W, H = pil_img.size
    x1, y1, x2, y2 = xyxy
    cx = (x1 + x2) / 2.0
    cy = (y1 + y2) / 2.0
    bw = x2 - x1
    bh = y2 - y1
    side = max(bw, bh) * scale

    # 최소 사이즈 보장
    side = max(side, MIN_BOX_SIDE_PX * 1.5)

    half = side / 2.0
    nx1 = int(round(cx - half))
    ny1 = int(round(cy - half))
    nx2 = int(round(cx + half))
    ny2 = int(round(cy + half))

    nx1 = max(0, nx1)
    ny1 = max(0, ny1)
    nx2 = min(W, nx2)
    ny2 = min(H, ny2)
    if nx2 <= nx1 or ny2 <= ny1:
        return None

    crop = pil_img.crop((nx1, ny1, nx2, ny2))
    return crop

def make_crops_with_yolo(img_path):
    """
    IMG_PATH 하나 받아서:
      - YOLO로 bbox 검출
      - 단일 + 거의 전체 차지면 full 이미지 1장
      - 그 외엔 square crop 여러 장
    결과: [PIL.Image, ...]
    """
    assert os.path.exists(img_path), f"이미지 없음: {img_path}"

    det = YOLO_MODEL(
        img_path,
        imgsz=YOLO_IMGSZ,
        conf=YOLO_CONF,
        iou=YOLO_IOU,
        device=YOLO_DEVICE,
        verbose=False
    )[0]

    img = Image.open(img_path).convert("RGB")
    W, H = img.size
    img_area = W * H

    raw_boxes = []
    if det.boxes is not None and len(det.boxes) > 0:
        for b in det.boxes.xyxy.cpu().numpy().tolist():
            x1, y1, x2, y2 = map(int, b)
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(W-1, x2)
            y2 = min(H-1, y2)
            if (x2-x1) >= MIN_BOX_SIDE_PX and (y2-y1) >= MIN_BOX_SIDE_PX:
                raw_boxes.append([x1, y1, x2, y2])

    # 단일 큰 박스면 전체 이미지 사용
    use_full_image_only = False
    if len(raw_boxes) == 1:
        x1, y1, x2, y2 = raw_boxes[0]
        box_area = (x2-x1) * (y2-y1)
        area_ratio = box_area / float(img_area + 1e-9)
        if area_ratio >= FULL_IMAGE_AREA_RATIO_THRESHOLD:
            use_full_image_only = True

    crop_images = []

    if use_full_image_only:
        crop_images.append(img.copy())
    else:
        for bbox in raw_boxes:
            sq = square_crop_from_bbox(img, bbox, scale=SQUARE_SCALE) if raw_boxes else None
            if sq is None:
                continue
            crop_images.append(sq)

    # YOLO가 아무것도 못 잡았으면 전체 이미지 fallback
    if not crop_images:
        crop_images.append(img.copy())

    return crop_images

# -------------------------------------------------
# 4) 엔드투엔드 추론 함수 (API로 쓰기 좋게)
# -------------------------------------------------
def infer_pill_image(img_path: str, topk_global: int = 5):
    """
    하나의 이미지 경로(img_path)를 받아:
      - YOLO로 소프트 크롭 여러 개 생성
      - 각 크롭을 ResNet(1324)로 추론
      - class별 max-pooling으로 전체 이미지 기준 최종 Top-K 계산
      - 최종 결과를 리스트로 return

    return 형식:
      [
        {"rank": 1, "idx": <class_idx>, "dl_idx": "<dl_idx>", "prob": <float>},
        ...
      ]
    """
    # 1) 크롭 생성
    crops = make_crops_with_yolo(img_path)

    # 2) 크롭별 Top-5 (전역 RESNET_MODEL 사용)
    batch_results = predict_resnet_batch(crops, topk=topk_global)

    # 3) 클래스별 max-pooling
    agg_scores = {}  # class_idx → max_prob
    for crop_res in batch_results:
        for t in crop_res:
            idx = t["idx"]
            p   = t["prob"]
            if idx not in agg_scores or p > agg_scores[idx]:
                agg_scores[idx] = p

    if not agg_scores:
        return []

    # 4) 전체 이미지 기준 최종 Top-K
    sorted_pairs = sorted(agg_scores.items(), key=lambda x: x[1], reverse=True)[:topk_global]

    final_results = []
    for rank, (idx, prob) in enumerate(sorted_pairs, start=1):
        dl_idx = idx_to_dl_idx(idx)
        final_results.append({
            "rank":  rank,
            "idx":   idx,
            "dl_idx": dl_idx,
            "prob":  prob,
        })

    return final_results

# -------------------------------------------------
# 5) 스크립트 직접 실행 시 동작 예시
# -------------------------------------------------
if __name__ == "__main__":
    assert os.path.exists(IMG_PATH), f"이미지 없음: {IMG_PATH}"
    results = infer_pill_image(IMG_PATH, topk_global=5)

    # ✅ API에 넘길 값은 이 results 그대로 쓰면 됨
    print("=== FINAL GLOBAL TOP-5 RESULTS ===")
    for r in results:
        print(
            f"Rank {r['rank']}: "
            f"class_idx={r['idx']}, dl_idx={r['dl_idx']}, prob={r['prob']*100:.2f}%"
        )
    # import json
    # print(json.dumps(results, ensure_ascii=False, indent=2))


Device: cpu
=== FINAL GLOBAL TOP-5 RESULTS ===
Rank 1: class_idx=1262, dl_idx=9271, prob=99.22%
Rank 2: class_idx=1070, dl_idx=18945, prob=2.75%
Rank 3: class_idx=1275, dl_idx=9562, prob=1.05%
Rank 4: class_idx=1308, dl_idx=38926, prob=0.64%
Rank 5: class_idx=1229, dl_idx=13553, prob=0.43%
